In [1]:
import sys
import pyspark
from pyspark.sql import SparkSession

In [2]:
sys.path.append("/opt/workspace")
from load_env import env_parameter

# Load from .env
env_vars = env_parameter()
spark_master = env_vars["spark_master"]
event_log_dir = env_vars["event_log_dir"]

In [3]:
# Start Spark session with PostgreSQL JDBC driver
spark = SparkSession.builder \
    .appName("SAP_BRONZE") \
    .master(spark_master) \
    .config("spark.jars.packages", ",".join([
        "org.apache.hadoop:hadoop-aws:3.3.2"
    ])) \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "admin") \
    .config("spark.hadoop.fs.s3a.secret.key", "password123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", event_log_dir) \
    .getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3af77f4e-0246-43f0-99a1-338386b1139a;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
downloading https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.2/hadoop-aws-3.3.2.jar ...
	[SUCCESSFUL ] org.apache.hadoop#hadoop-aws;3.3.2!hadoop-aws.jar (865ms)
downloading https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.11.1026/aws-java-sdk-bundle-1.11.1026.jar ...
	[SUCCESSFUL ] com.amazonaws#aws-java-sdk-bundle;1.11.1026!aws-java-sdk-bundle.jar (17387ms)
downloading https://repo1.maven.org/maven2/org/wildfly/openssl/wildfly-openssl/1.0.7.Final/wildfly-openssl-1.0.7.Final.jar ...
	[SUCCESSFUL ] org.wildf

In [4]:
df_landing = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("s3a://landing/adrc.csv")

25/07/16 16:53:12 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

In [7]:
df_landing.show(1,truncate=True,vertical=True)

-RECORD 0-------------------------------
 client           | 200                 
 addrnumber       | 15211               
 date_from        | 2001-01-01          
 nation           | NULL                
 date_to          | 9999-12-31          
 title            | NULL                
 name1            | Ada Burchette       
 name2            | NULL                
 name3            | NULL                
 name4            | NULL                
 name_text        | NULL                
 name_co          | NULL                
 city1            | New York            
 city2            | NULL                
 city_code        | NULL                
 cityp_code       | NULL                
 home_city        | NULL                
 cityh_code       | NULL                
 chckstatus       | NULL                
 regiogroup       | NULL                
 post_code1       | 10001               
 post_code2       | NULL                
 post_code3       | NULL                
 pcode1_ext     

In [9]:
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "LEGACY")

In [10]:
df_landing.write \
    .mode("overwrite") \
    .parquet("s3a://bronze/sap/adrc/")